<a href="https://colab.research.google.com/github/santoshkumarbvp/Deep-Learning/blob/main/Vgg19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Wed Jan  6 17:14:07 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.27.04    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8    12W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import tensorflow as tf 

if tf.test.gpu_device_name(): 
    

    print('Default GPU Device:{}'.format(tf.test.gpu_device_name()))

else:

    print("Please install GPU version of TF")

Default GPU Device:/device:GPU:0


In [ ]:
tf.test.gpu_device_name()


'/device:GPU:0'

In [5]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob


In [ ]:
import tensorflow as tf
tf.__version__

'2.4.0'

In [3]:
# re-size all the images to this
IMAGE_SIZE = [224, 224]

train_path = '/content/drive/MyDrive/data/train/'
valid_path = '/content/drive/MyDrive/data/test/'

In [6]:
vgg19 = VGG19(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

80142336/80134624 [==============================] - 1s 0us/step


In [7]:
# don't train existing weights
for layer in vgg19.layers:
    layer.trainable = False

In [9]:
folders = glob(train_path+'/*')
folders

['/content/drive/MyDrive/data/train/dog',
 '/content/drive/MyDrive/data/train/cat']

In [13]:
x = Flatten()(vgg19.output)

In [16]:
prediction = Dense(len(folders), activation='sigmoid')(x)

# create a model object
model = Model(inputs=vgg19.input, outputs=prediction)

In [17]:
glob(valid_path+'/*')

['/content/drive/MyDrive/data/test/cat',
 '/content/drive/MyDrive/data/test/dog']

In [18]:

# view the structure of the model
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [19]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)


In [20]:
import os
import shutil

os.listdir(train_path) #First find where the ".ipynb_checkpoints" is located.



['dog', 'cat']

In [ ]:
#shutil.rmtree("/content/sample_data/data/train/.ipynb_checkpoints") #be careful with shutil.rmtree() because it deletes every tree in that path. In other words, do not make mistakes.

In [ ]:
#shutil.rmtree("/content/sample_data/data/test/.ipynb_checkpoints")

In [21]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [22]:
training_set = train_datagen.flow_from_directory(train_path,
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 4002 images belonging to 2 classes.


In [23]:
test_set = test_datagen.flow_from_directory(valid_path,
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 1002 images belonging to 2 classes.


In [24]:
from tensorflow.keras.callbacks import EarlyStopping

In [25]:
es = EarlyStopping(monitor='val_loss',patience=5)

In [ ]:
model.fit(
  training_set,
  validation_data=test_set,
  epochs=20,
  callbacks=[es],
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

Epoch 1/20
 58/126 [============>.................] - ETA: 15:47 - loss: 1.1342 - accuracy: 0.6208

In [ ]:
model.save('/content/drive/MyDrive/data/mymodel_vgg19.h5')

In [ ]:
img=image.load_img('/content/drive/MyDrive/data/test/cat/cat.2003.jpg',target_size=(224,224))

x=image.img_to_array(img)
x=x/255
x=np.expand_dims(x,axis=0)
img_data=preprocess_input(x)
model.predict(img_data).round(2)

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score


In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
from tensorflow.keras.preprocessing import image

In [ ]:
model_new = load_model('/content/drive/MyDrive/data/mymodel.h5')

In [ ]:
import os

In [ ]:
# check for all cats
dirs  = os.listdir('/content/drive/MyDrive/data/test/cat/')
all_files = []
for image_name in dirs:
  img = image.load_img('/content/drive/MyDrive/data/test/cat/'+image_name,target_size=(224,224))
  all_files.append(image.img_to_array(img))


KeyboardInterrupt: ignored

In [ ]:
test_cat=np.array(all_files)

In [ ]:
test_cat=test_cat/255

In [ ]:
pred_cat=model.predict(test_cat)

In [ ]:
pred_cat_i= [np.argmax(i) for i in pred_cat]

In [ ]:
np.array(pred_cat_i).sum()

32

In [ ]:
# check for all dogs
dirs  = os.listdir('/content/drive/MyDrive/data/test/dog/')
all_files = []
for image_name in dirs:
  img = image.load_img('/content/drive/MyDrive/data/test/dog/'+image_name,target_size=(224,224))
  all_files.append(image.img_to_array(img))


test_dog=np.array(all_files)
test_dog=test_dog/255

pred_dog=model.predict(test_dog)

pred_dog_i= [np.argmax(i) for i in pred_dog]
np.array(pred_dog_i).sum()

In [ ]:
training_set.class_indices

{'cat': 0, 'dog': 1}

In [ ]:
model.predict(img_data).round(2)

In [ ]:
img=image.load_img('/content/drive/MyDrive/data/test/cat/cat.2081.jpg',target_size=(224,224))

x=image.img_to_array(img)
x=x/255
x=np.expand_dims(x,axis=0)
# img_data=preprocess_input(x)
model.predict(x).round(2)

array([[1., 0.]], dtype=float32)